In [183]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.parser import parse

In [184]:
FILEPATH_ALL = "all_data.csv"
FILEPATH_2020 = "with_closed_ations_as________select_app_2020_X5_STORES.csv"
FILEPATH_2019 = "with_closed_ations_as________select_app_2019_X5_STORES.csv"
FILEPATH_2018 = "with_closed_ations_as________select_app_2018_X5_STORES.csv"
df = pd.read_csv(FILEPATH_2020)

/home/lev/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [185]:
# Это скрипт для объединения датасетов в один + удаления ненужной инфы + фикса дат
# df18 = pd.read_csv(FILEPATH_2018)
# df19 = pd.read_csv(FILEPATH_2019)
# df20 = pd.read_csv(FILEPATH_2020)
# df_all = pd.concat([df18, df19, df20])
# df_all = df_all.reset_index(drop=True)
# df_all = df_all[df_all['position_name'] == 'Продавец-кассир']
# df_all = df_all[df_all['is_active_action'] == True]
# df_all = df_all.drop('is_active_action', axis=1)
# df_all['action_date'] = df_all['action_date'].str[:19]
# df_all['action_date'] = pd.to_datetime(df_all['action_date'], format="%Y-%m-%d %H:%M:%S")
# df_all = df_all.reset_index(drop=True)
# df_all.to_csv('all_data.csv')

In [186]:
df = df[df['position_name'] == 'Продавец-кассир']
df = df[df['is_active_action'] == True]
df = df.drop('is_active_action', axis=1)
df['action_date'] = df['action_date'].str[:19]
df['action_date'] = pd.to_datetime(df['action_date'], format="%Y-%m-%d %H:%M:%S")

In [187]:
def generate_unique_csv(df, filename):
    df_new = df[['stage_name','activity_name','result_name']]
    df_new = df_new.drop_duplicates().sort_values('stage_name')
    df_new = df_new.reset_index(drop=True)
    df_new
    df_copy = df.copy()
    df_copy[['stage_name', 'activity_name', 'result_name']] = df_copy[['stage_name', 'activity_name', 'result_name']].fillna('')
    df_new[['stage_name', 'activity_name', 'result_name']] = df_new[['stage_name', 'activity_name', 'result_name']].fillna('')
    df_new['count'] = 0
    for index, row in df_new.iterrows():
        val = df_copy[(df_copy['stage_name'] == row['stage_name']) & (df_copy['activity_name'] == row['activity_name']) & (df_copy['result_name'] == row['result_name'])].shape[0]
        df_new.at[index, 'count'] = val
    df_new.sort_values('stage_name').to_csv(filename)
    return df_new

def clientInfo(df, id):
    return df[df['id_applicant'] == id].drop(['is_open_action', 'position_name'], axis=1).sort_values('action_date')

In [188]:
df_hired = df[(df['stage_name'] == 'Оформление') & (df['activity_name'] == 'Узнать результаты приема') & (df['result_name'] == 'Принят')]

In [189]:
df_hired = df_hired.reset_index(drop=True)
df_hired

,id_applicant,id_partner,id_log,department_id,action_date,stage_name,activity_name,result_name,position_name,is_open_action
0,4774172,3835783,26666690,31992,2020-03-26 14:48:20,Оформление,Узнать результаты приема,Принят,Продавец-кассир,False
1,4813502,3821610,26668701,18141,2020-03-26 15:39:35,Оформление,Узнать результаты приема,Принят,Продавец-кассир,False
2,4813678,2057903,26669783,31462,2020-03-26 16:01:07,Оформление,Узнать результаты приема,Принят,Продавец-кассир,False
3,4813843,3822462,26671477,30354,2020-03-26 16:21:38,Оформление,Узнать результаты приема,Принят,Продавец-кассир,False
4,4766031,3831548,26674667,31760,2020-03-26 17:16:26,Оформление,Узнать результаты приема,Принят,Продавец-кассир,False
...,...,...,...,...,...,...,...,...,...,...
7038,4854560,3855417,26915732,49671,2020-04-07 11:05:40,Оформление,Узнать результаты приема,Принят,Продавец-кассир,False
7039,4883157,3860226,27087224,10660,2020-04-13 15:18:28,Оформление,Узнать результаты приема,Принят,Продавец-кассир,False
7040,4846293,3856487,27087777,7649,2020-04-13 15:32:12,Оформление,Узнать результаты приема,Принят,Продавец-кассир,False
7041,4864742,1593647,27105932,23665,2020-04-14 11:07:42,Оформление,Узнать результаты приема,Принят,Продавец-кассир,False


In [190]:
hired_ids = df_hired['id_applicant']

In [191]:
hired_ids

0       4774172
1       4813502
2       4813678
3       4813843
4       4766031
         ...   
7038    4854560
7039    4883157
7040    4846293
7041    4864742
7042    4786759
Name: id_applicant, Length: 7043, dtype: int64

In [192]:
df_filtered = df[df['id_applicant'].isin(hired_ids)]

In [193]:
df_created = df_filtered[df_filtered['stage_name'] == 'Создание кандидата']
df_created

,id_applicant,id_partner,id_log,department_id,action_date,stage_name,activity_name,result_name,position_name,is_open_action
21,4808927,3831491,-1,11904,2020-03-25 16:56:43,Создание кандидата,NaN,NaN,Продавец-кассир,False
54,4777845,1337824,-1,31597,2020-03-18 14:45:40,Создание кандидата,NaN,NaN,Продавец-кассир,False
63,4775332,3836431,-1,35802,2020-03-18 10:30:14,Создание кандидата,NaN,NaN,Продавец-кассир,False
64,4753541,3817503,-1,5725,2020-03-12 14:20:55,Создание кандидата,NaN,NaN,Продавец-кассир,False
76,4758922,3543341,-1,19047,2020-03-13 13:46:17,Создание кандидата,NaN,NaN,Продавец-кассир,False
...,...,...,...,...,...,...,...,...,...,...
214750,4744437,3820403,-1,25329,2020-03-11 09:38:27,Создание кандидата,NaN,NaN,Продавец-кассир,False
214752,4770238,3833787,-1,17151,2020-03-17 11:14:48,Создание кандидата,NaN,NaN,Продавец-кассир,False
214758,4808857,3758979,-1,10962,2020-03-25 16:47:15,Создание кандидата,NaN,NaN,Продавец-кассир,False
214760,4733920,3813915,-1,45183,2020-03-06 13:13:51,Создание кандидата,NaN,NaN,Продавец-кассир,False


In [194]:
created_ids = df_created['id_applicant']
created_ids.count()

7042

In [195]:
ids = set(created_ids).intersection(set(hired_ids))
len(ids)

7042

In [196]:
df_filtered = df[df['id_applicant'].isin(hired_ids)]
df_filtered

,id_applicant,id_partner,id_log,department_id,action_date,stage_name,activity_name,result_name,position_name,is_open_action
21,4808927,3831491,-1,11904,2020-03-25 16:56:43,Создание кандидата,NaN,NaN,Продавец-кассир,False
54,4777845,1337824,-1,31597,2020-03-18 14:45:40,Создание кандидата,NaN,NaN,Продавец-кассир,False
63,4775332,3836431,-1,35802,2020-03-18 10:30:14,Создание кандидата,NaN,NaN,Продавец-кассир,False
64,4753541,3817503,-1,5725,2020-03-12 14:20:55,Создание кандидата,NaN,NaN,Продавец-кассир,False
76,4758922,3543341,-1,19047,2020-03-13 13:46:17,Создание кандидата,NaN,NaN,Продавец-кассир,False
...,...,...,...,...,...,...,...,...,...,...
732430,4817687,3827588,-1,12807,2020-04-23 16:26:11,Оформление,Предоставить обратную связь по кандидату,NaN,Продавец-кассир,True
732533,4794055,3223422,-1,9226,2020-04-23 16:26:11,Оформление,Предоставить обратную связь по кандидату,NaN,Продавец-кассир,True
732546,4824789,3861710,-1,30408,2020-04-23 16:26:11,Оформление,Предоставить обратную связь по кандидату,NaN,Продавец-кассир,True
732600,4815525,3857156,-1,12949,2020-04-23 16:26:11,Оформление,Предоставить обратную связь по кандидату,NaN,Продавец-кассир,True


In [197]:
df_filtered = df_filtered.sort_values(['id_applicant','action_date'])
df_filtered

,id_applicant,id_partner,id_log,department_id,action_date,stage_name,activity_name,result_name,position_name,is_open_action
214486,4554398,3712663,-1,34016,2020-01-27 16:30:47,Создание кандидата,NaN,NaN,Продавец-кассир,False
610929,4554398,3712663,26634580,34016,2020-03-25 16:13:10,Оформление,Узнать результаты приема,Принят,Продавец-кассир,False
63269,4578838,3732215,-1,31117,2020-01-31 16:15:32,Создание кандидата,NaN,NaN,Продавец-кассир,False
348409,4578838,3732215,26847382,31117,2020-04-03 17:06:12,Оформление,Узнать результаты приема,Принят,Продавец-кассир,False
58516,4579888,2776525,-1,31475,2020-02-01 07:29:11,Создание кандидата,NaN,NaN,Продавец-кассир,False
...,...,...,...,...,...,...,...,...,...,...
448347,4886615,1040329,27106030,27473,2020-04-14 11:09:14,СБ,Подготовить анкету кандидата,Подготовлена,Продавец-кассир,False
448348,4886615,1040329,27106031,27473,2020-04-14 11:09:15,СБ,Узнать результаты СБ,Подходит,Продавец-кассир,False
448349,4886615,1040329,27106032,27473,2020-04-14 11:09:16,СБ,Проверка формы обращения,Не резерв,Продавец-кассир,False
448350,4886615,1040329,27106033,27473,2020-04-14 11:09:18,Оформление,Назначить время на оформление,Время назначено,Продавец-кассир,False


In [198]:
groups = df_filtered.groupby('id_applicant')

In [199]:
last = groups.last()
last

,id_partner,id_log,department_id,action_date,stage_name,activity_name,result_name,position_name,is_open_action
id_applicant,,,,,,,,,
4554398,3712663,26634580,34016,2020-03-25 16:13:10,Оформление,Узнать результаты приема,Принят,Продавец-кассир,False
4578838,3732215,26847382,31117,2020-04-03 17:06:12,Оформление,Узнать результаты приема,Принят,Продавец-кассир,False
4579888,2776525,26846233,31475,2020-04-03 16:40:06,Оформление,Узнать результаты приема,Принят,Продавец-кассир,False
4592572,2542953,26895620,12874,2020-04-06 15:12:24,Оформление,Узнать результаты приема,Принят,Продавец-кассир,False
4594868,3275293,26795536,36541,2020-04-01 16:07:24,Оформление,Узнать результаты приема,Принят,Продавец-кассир,False
...,...,...,...,...,...,...,...,...,...
4886560,1197774,27105784,35106,2020-04-14 11:04:44,Оформление,Узнать результаты приема,Принят,Продавец-кассир,False
4886575,1542748,27105875,28162,2020-04-14 11:06:42,Оформление,Узнать результаты приема,Принят,Продавец-кассир,False
4886579,3878216,27105890,10530,2020-04-14 11:07:06,Оформление,Узнать результаты приема,Принят,Продавец-кассир,False


In [200]:
last['activity_name'].unique()

array(['Узнать результаты приема',
       'Предоставить обратную связь по кандидату'], dtype=object)

In [201]:
last['stage_name'].unique()

array(['Оформление', 'Выход на работу'], dtype=object)

In [202]:
last['result_name'].unique()

array(['Принят', 'Отказ руководителя',
       'ОС не предоставлена руководителем', 'Отказ кандидата', 'Недозвон'],
      dtype=object)

In [203]:
not_hired = last[last['result_name'] != 'Принят']
# not_hired

In [204]:
# clientInfo(df,4766325)

In [205]:
df_filtered = df_filtered[df_filtered['id_applicant'].isin(ids_not_hired) == False]
df_filtered

,id_applicant,id_partner,id_log,department_id,action_date,stage_name,activity_name,result_name,position_name,is_open_action
214486,4554398,3712663,-1,34016,2020-01-27 16:30:47,Создание кандидата,NaN,NaN,Продавец-кассир,False
610929,4554398,3712663,26634580,34016,2020-03-25 16:13:10,Оформление,Узнать результаты приема,Принят,Продавец-кассир,False
63269,4578838,3732215,-1,31117,2020-01-31 16:15:32,Создание кандидата,NaN,NaN,Продавец-кассир,False
348409,4578838,3732215,26847382,31117,2020-04-03 17:06:12,Оформление,Узнать результаты приема,Принят,Продавец-кассир,False
58516,4579888,2776525,-1,31475,2020-02-01 07:29:11,Создание кандидата,NaN,NaN,Продавец-кассир,False
...,...,...,...,...,...,...,...,...,...,...
448347,4886615,1040329,27106030,27473,2020-04-14 11:09:14,СБ,Подготовить анкету кандидата,Подготовлена,Продавец-кассир,False
448348,4886615,1040329,27106031,27473,2020-04-14 11:09:15,СБ,Узнать результаты СБ,Подходит,Продавец-кассир,False
448349,4886615,1040329,27106032,27473,2020-04-14 11:09:16,СБ,Проверка формы обращения,Не резерв,Продавец-кассир,False
448350,4886615,1040329,27106033,27473,2020-04-14 11:09:18,Оформление,Назначить время на оформление,Время назначено,Продавец-кассир,False


In [289]:
groups = df_filtered.groupby('id_applicant')

In [290]:
df_new = pd.DataFrame()
df_new['id'] = list(groups.groups)
df_new['actions'] = groups['action_date'].count().values
df_new['first'] = groups.first()['action_date'].values
df_new['last'] = groups.last()['action_date'].values
# df_new['last_action'] = groups.last()['action_date'].values

In [291]:
df_new

,id,actions,first,last
0,4554398,2,2020-01-27 16:30:47,2020-03-25 16:13:10
1,4578838,2,2020-01-31 16:15:32,2020-04-03 17:06:12
2,4579888,2,2020-02-01 07:29:11,2020-04-03 16:40:06
3,4592572,12,2020-02-04 15:16:52,2020-04-06 15:12:24
4,4594868,3,2020-02-05 09:08:35,2020-04-01 16:07:24
...,...,...,...,...
7019,4886560,10,2020-04-14 11:04:14,2020-04-14 11:04:44
7020,4886575,10,2020-04-14 11:06:05,2020-04-14 11:06:42
7021,4886579,10,2020-04-14 11:06:30,2020-04-14 11:07:06
7022,4886599,10,2020-04-14 11:07:46,2020-04-14 11:08:17


In [292]:
df_new['diff'] = df_new['last'] - df_new['first']

In [293]:
df_new

,id,actions,first,last,diff
0,4554398,2,2020-01-27 16:30:47,2020-03-25 16:13:10,57 days 23:42:23
1,4578838,2,2020-01-31 16:15:32,2020-04-03 17:06:12,63 days 00:50:40
2,4579888,2,2020-02-01 07:29:11,2020-04-03 16:40:06,62 days 09:10:55
3,4592572,12,2020-02-04 15:16:52,2020-04-06 15:12:24,61 days 23:55:32
4,4594868,3,2020-02-05 09:08:35,2020-04-01 16:07:24,56 days 06:58:49
...,...,...,...,...,...
7019,4886560,10,2020-04-14 11:04:14,2020-04-14 11:04:44,0 days 00:00:30
7020,4886575,10,2020-04-14 11:06:05,2020-04-14 11:06:42,0 days 00:00:37
7021,4886579,10,2020-04-14 11:06:30,2020-04-14 11:07:06,0 days 00:00:36
7022,4886599,10,2020-04-14 11:07:46,2020-04-14 11:08:17,0 days 00:00:31


In [294]:
df_new['diff_days'] = df_new['diff'].apply(lambda x: x.days)
df_new['diff_hours'] = df_new['diff'].apply(lambda x: x.total_seconds()//3600)

In [295]:
df_new

,id,actions,first,last,diff,diff_days,diff_hours
0,4554398,2,2020-01-27 16:30:47,2020-03-25 16:13:10,57 days 23:42:23,57,1391.0
1,4578838,2,2020-01-31 16:15:32,2020-04-03 17:06:12,63 days 00:50:40,63,1512.0
2,4579888,2,2020-02-01 07:29:11,2020-04-03 16:40:06,62 days 09:10:55,62,1497.0
3,4592572,12,2020-02-04 15:16:52,2020-04-06 15:12:24,61 days 23:55:32,61,1487.0
4,4594868,3,2020-02-05 09:08:35,2020-04-01 16:07:24,56 days 06:58:49,56,1350.0
...,...,...,...,...,...,...,...
7019,4886560,10,2020-04-14 11:04:14,2020-04-14 11:04:44,0 days 00:00:30,0,0.0
7020,4886575,10,2020-04-14 11:06:05,2020-04-14 11:06:42,0 days 00:00:37,0,0.0
7021,4886579,10,2020-04-14 11:06:30,2020-04-14 11:07:06,0 days 00:00:36,0,0.0
7022,4886599,10,2020-04-14 11:07:46,2020-04-14 11:08:17,0 days 00:00:31,0,0.0


In [296]:
df_new['diff_hours'].describe()

count    7024.000000
mean      151.703730
std       175.655761
min         0.000000
25%         0.000000
50%       121.000000
75%       232.250000
max      1512.000000
Name: diff_hours, dtype: float64

In [297]:
df_new = df_new[df_new['diff_hours'] > 0]
df_new

,id,actions,first,last,diff,diff_days,diff_hours
0,4554398,2,2020-01-27 16:30:47,2020-03-25 16:13:10,57 days 23:42:23,57,1391.0
1,4578838,2,2020-01-31 16:15:32,2020-04-03 17:06:12,63 days 00:50:40,63,1512.0
2,4579888,2,2020-02-01 07:29:11,2020-04-03 16:40:06,62 days 09:10:55,62,1497.0
3,4592572,12,2020-02-04 15:16:52,2020-04-06 15:12:24,61 days 23:55:32,61,1487.0
4,4594868,3,2020-02-05 09:08:35,2020-04-01 16:07:24,56 days 06:58:49,56,1350.0
...,...,...,...,...,...,...,...
6647,4874547,10,2020-04-10 17:10:11,2020-04-13 16:47:34,2 days 23:37:23,2,71.0
6648,4874561,10,2020-04-10 17:12:31,2020-04-13 16:45:57,2 days 23:33:26,2,71.0
6655,4874899,10,2020-04-10 18:37:18,2020-04-13 17:18:34,2 days 22:41:16,2,70.0
6656,4875075,12,2020-04-11 08:31:29,2020-04-13 13:00:37,2 days 04:29:08,2,52.0


In [298]:
df_new[df_new['diff_hours'] < 24]

,id,actions,first,last,diff,diff_days,diff_hours
2319,4810407,12,2020-03-26 09:45:22,2020-03-26 16:27:54,0 days 06:42:32,0,6.0
2433,4812269,12,2020-03-26 12:42:49,2020-03-26 14:41:07,0 days 01:58:18,0,1.0
2478,4812633,13,2020-03-26 13:44:41,2020-03-27 12:10:15,0 days 22:25:34,0,22.0
3371,4821374,12,2020-03-30 11:19:00,2020-03-30 17:12:47,0 days 05:53:47,0,5.0
3394,4821649,12,2020-03-30 11:50:40,2020-03-30 16:01:49,0 days 04:11:09,0,4.0
3397,4821716,13,2020-03-30 11:58:37,2020-03-31 10:41:42,0 days 22:43:05,0,22.0
4229,4828397,12,2020-03-31 15:49:42,2020-03-31 17:17:03,0 days 01:27:21,0,1.0
4390,4829816,12,2020-04-01 09:43:32,2020-04-01 16:05:31,0 days 06:21:59,0,6.0
5502,4851011,10,2020-04-06 15:25:52,2020-04-07 12:36:32,0 days 21:10:40,0,21.0
6059,4858379,13,2020-04-07 17:28:53,2020-04-08 17:17:16,0 days 23:48:23,0,23.0


In [299]:
df_new = df_new[df_new['diff_hours'] >= 24]
df_new

,id,actions,first,last,diff,diff_days,diff_hours
0,4554398,2,2020-01-27 16:30:47,2020-03-25 16:13:10,57 days 23:42:23,57,1391.0
1,4578838,2,2020-01-31 16:15:32,2020-04-03 17:06:12,63 days 00:50:40,63,1512.0
2,4579888,2,2020-02-01 07:29:11,2020-04-03 16:40:06,62 days 09:10:55,62,1497.0
3,4592572,12,2020-02-04 15:16:52,2020-04-06 15:12:24,61 days 23:55:32,61,1487.0
4,4594868,3,2020-02-05 09:08:35,2020-04-01 16:07:24,56 days 06:58:49,56,1350.0
...,...,...,...,...,...,...,...
6552,4871932,13,2020-04-10 11:38:24,2020-04-13 14:14:22,3 days 02:35:58,3,74.0
6647,4874547,10,2020-04-10 17:10:11,2020-04-13 16:47:34,2 days 23:37:23,2,71.0
6648,4874561,10,2020-04-10 17:12:31,2020-04-13 16:45:57,2 days 23:33:26,2,71.0
6655,4874899,10,2020-04-10 18:37:18,2020-04-13 17:18:34,2 days 22:41:16,2,70.0


In [300]:
df_new['work_days'] = df_new.apply(lambda x: np.busday_count(x['first'].strftime('%Y-%m-%d'), x['last'].strftime('%Y-%m-%d')), axis=1)

<ipython-input-300-0e56b01a1201>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['work_days'] = df_new.apply(lambda x: np.busday_count(x['first'].strftime('%Y-%m-%d'), x['last'].strftime('%Y-%m-%d')), axis=1)


In [301]:
df_new

,id,actions,first,last,diff,diff_days,diff_hours,work_days
0,4554398,2,2020-01-27 16:30:47,2020-03-25 16:13:10,57 days 23:42:23,57,1391.0,42
1,4578838,2,2020-01-31 16:15:32,2020-04-03 17:06:12,63 days 00:50:40,63,1512.0,45
2,4579888,2,2020-02-01 07:29:11,2020-04-03 16:40:06,62 days 09:10:55,62,1497.0,44
3,4592572,12,2020-02-04 15:16:52,2020-04-06 15:12:24,61 days 23:55:32,61,1487.0,44
4,4594868,3,2020-02-05 09:08:35,2020-04-01 16:07:24,56 days 06:58:49,56,1350.0,40
...,...,...,...,...,...,...,...,...
6552,4871932,13,2020-04-10 11:38:24,2020-04-13 14:14:22,3 days 02:35:58,3,74.0,1
6647,4874547,10,2020-04-10 17:10:11,2020-04-13 16:47:34,2 days 23:37:23,2,71.0,1
6648,4874561,10,2020-04-10 17:12:31,2020-04-13 16:45:57,2 days 23:33:26,2,71.0,1
6655,4874899,10,2020-04-10 18:37:18,2020-04-13 17:18:34,2 days 22:41:16,2,70.0,1


In [302]:
df_new[df_new['work_days'] == 0]

,id,actions,first,last,diff,diff_days,diff_hours,work_days
3159,4819113,12,2020-03-29 11:00:13,2020-03-30 20:26:14,1 days 09:26:01,1,33.0,0
5219,4844085,10,2020-04-04 11:35:27,2020-04-06 17:27:09,2 days 05:51:42,2,53.0,0
5234,4844358,12,2020-04-04 12:25:10,2020-04-06 14:26:12,2 days 02:01:02,2,50.0,0
6656,4875075,12,2020-04-11 08:31:29,2020-04-13 13:00:37,2 days 04:29:08,2,52.0,0


In [310]:
df_new.loc[(df_new['work_days'] == 0), 'work_days'] = 1

/home/lev/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [311]:
df_new[df_new['work_days'] == 0]

,id,actions,first,last,diff,diff_days,diff_hours,work_days


In [312]:
df_new.describe()

,id,actions,diff,diff_days,diff_hours,work_days
count,4.544000e+03,4544.000000,4544,4544.000000,4544.000000,4544.000000
mean,4.806344e+06,9.951585,9 days 18:57:05.819322183,9.419674,234.459947,6.893046
std,3.579556e+04,4.772466,7 days 00:13:06.754435005,7.019464,168.215699,5.004599
min,4.554398e+06,2.000000,1 days 00:41:00,1.000000,24.000000,1.000000
25%,4.787526e+06,6.000000,5 days 04:19:28,5.000000,124.000000,4.000000
50%,4.811888e+06,11.000000,7 days 22:44:55,7.000000,190.000000,6.000000
75%,4.830332e+06,13.000000,12 days 14:23:42,12.000000,301.750000,9.000000
max,4.875075e+06,28.000000,63 days 00:50:40,63.000000,1512.000000,45.000000
